In [7]:
import matplotlib.pyplot as plt
import numpy as np
import math


def direct_estimator():
    c_list = [i / 100 for i in range(14, 3, -1)]
    max_attemps = 1000000
    relative_error_percent = 5
    for c in c_list:
        calculated_p_c = 0.5 * (1 - math.erf(1 / math.sqrt(2 * c)))
        # Lets take relative error 5%
        low_limit = (1 - relative_error_percent / 100) * calculated_p_c
        high_limit = (1 + relative_error_percent / 100) * calculated_p_c
        greater_than_1 = 0
        for m in range(1, max_attemps):
            experimental_p_c = greater_than_1 / m
            if low_limit <= experimental_p_c <= high_limit:
                break
            w = 0
            for n in range(1, 500):
                xi = np.random.normal(0, 1)
                z = (n - 0.5) * np.pi
                w += xi * np.sin(z * c) / z
            w *= math.sqrt(2)
            if w > 1:
                greater_than_1 += 1
        if m < max_attemps - 1:
            print(f'It took {m} attempts for {c} to reach the relative error of 5%')
        else:
            print(f'For {c} no successful events happened in {max_attemps} attempts')
            break

In [8]:
direct_estimator()

It took 760 attempts for 0.14 to reach the relative error of 5%
It took 1117 attempts for 0.13 to reach the relative error of 5%
It took 979 attempts for 0.12 to reach the relative error of 5%
It took 742 attempts for 0.11 to reach the relative error of 5%
It took 16478 attempts for 0.1 to reach the relative error of 5%
It took 2220 attempts for 0.09 to reach the relative error of 5%
It took 4681 attempts for 0.08 to reach the relative error of 5%
It took 12129 attempts for 0.07 to reach the relative error of 5%
It took 42749 attempts for 0.06 to reach the relative error of 5%
It took 245960 attempts for 0.05 to reach the relative error of 5%
For 0.04 no successful events happened in 1000000 attempts


In [ ]:
# We see that the number of required attempts grows exponentially as c becomes smaller. 
# For c=0.05 we couldn't even estimate the probability, as we didn't have a single successful event in 1000000 attempts
# That makes sense sinse the probability for c=0.04 is 0.5*(1-erf(1/sqrt(2*c))) = 2.87*10^(-7)